<a href="https://colab.research.google.com/github/gorzanskik-ai/short-term-stocks-price-prediction/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='LOSS vs. VAL_LOSS', xaxis_title='Epoki', yaxis_title='LOSS')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root mean squared error')
    fig.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import plotly.express as px

np.random.seed(42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, LSTM
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path1 = "/content/drive/MyDrive/master's thesis/day/regression1.csv"
df_raw1 = pd.read_csv(path1)
df_raw1.index = df_raw1.Date
df_raw1 = df_raw1.drop('Date', axis=1)
df1 = df_raw1.copy()
df1 = df1[['Open', 'High', 'Low', 'Close', 'Volume', 'Return', 'Volatility', 'LH % Change', 'OC % Change', 'Candlestick', 'TARGET']]

path2 = "/content/drive/MyDrive/master's thesis/day/regression.csv"
df_raw2 = pd.read_csv(path2)
df_raw2.index = df_raw2.Date
df_raw2 = df_raw2.drop('Date', axis=1)
df2 = df_raw2.copy()

In [ ]:
df2

,Open,High,Low,Close,Volume,Day,Month,Year,OC % Change,LH % Change,...,n-4,n-5,RSI,MFI,CMO,UO,ADX,TARGET,Return,Volatility
Date,,,,,,,,,,,,,,,,,,,,,
1950-02-03,17.29,17.29,17.29,17.29,0.000000,3,2,1950,0.000000,0.000000,...,0.000000,0.000000,72.851230,70.041111,45.702460,88.054312,21.934749,17.35,0.000000,0.000000
1950-02-04,17.35,17.35,17.35,17.35,0.000000,4,2,1950,0.000000,0.000000,...,0.682610,0.682610,74.709357,71.095011,49.418714,88.504359,24.860554,17.32,0.000000,0.000000
1950-02-06,17.32,17.32,17.32,17.32,-1.095658,6,2,1950,0.000000,0.000000,...,0.547500,0.721815,72.053911,61.751078,44.107822,83.370800,26.840183,17.23,0.000000,0.000000
1950-02-07,17.23,17.23,17.23,17.23,-0.938628,7,2,1950,0.000000,0.000000,...,1.385865,0.590286,64.631997,54.625423,29.263994,68.776487,27.081882,17.21,0.000000,0.000000
1950-02-08,17.21,17.21,17.21,17.21,-0.590611,8,2,1950,0.000000,0.000000,...,1.214904,1.426948,63.077098,54.773970,26.154196,65.593003,26.991920,17.28,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22,4385.83,4385.83,4267.62,4271.78,0.193298,22,4,2022,-0.026004,0.027699,...,-0.559223,-0.577540,36.919811,42.757885,-26.160379,35.280596,16.275124,4296.12,-1.786300,1.582911
2022-04-25,4255.34,4299.02,4200.82,4296.12,1.013410,25,4,2022,0.009583,0.023376,...,0.099049,-0.559223,39.271922,35.524350,-21.456156,39.132563,17.917140,4175.20,0.878777,0.843698
2022-04-26,4278.14,4278.14,4175.04,4175.20,0.448339,26,4,2022,-0.024062,0.024694,...,0.069771,0.099049,32.740441,35.114347,-34.519118,35.796959,19.639045,4183.96,-1.470713,0.945241


Train Test Split

In [ ]:
X = df2.drop('TARGET', axis=1).copy()
y = df2['TARGET'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

features = ['Day', 'Month', 'Year', 'RSI', 'MFI', 'CMO', 'UO', 'ADX'] 
scaler = MinMaxScaler()
scaler.fit(X_train[features])
X_train[features] = scaler.transform(X_train[features])
X_test[features] = scaler.transform(X_test[features])

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

y_train = y_train.reshape(y_train.shape[0], 1, 1)

In [ ]:
def build_model_rnn():
    lstm = Sequential()
    lstm.add(LSTM(32, input_shape=(1, X_train.shape[2]), activation='relu', return_sequences=False))
    lstm.add(Dense(1))
    lstm.compile(optimizer='adam', 
                 loss='mape',
                 metrics=['mape', 'mse'])

    return lstm

In [ ]:
model = build_model_rnn()
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 32)                7168      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#es = EarlyStopping(monitor='val_mse', mode='min', verbose=0, patience=1)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.25, verbose=0, batch_size=32) #, callbacks=[es])

metrics = pd.DataFrame(history.history)
metrics.iloc[-1:]

,loss,mape,mse,val_loss,val_mape,val_mse
19,0.711244,0.711244,167.119385,0.695453,0.695453,190.5457


In [ ]:
y_pred = model.predict(X_test).flatten()
y_true = np.copy(y_test)

y_pred = y_pred.reshape(y_pred.shape[0])

print('mape', mean_absolute_percentage_error(y_true, y_pred))
print('rmse', mean_squared_error(y_true, y_pred, squared=False))

mape 0.0067178862437627706
rmse 13.818907853404939


In [ ]:
model.save("/content/drive/MyDrive/master's thesis/day/regression.h5")

In [ ]:
plot_history(history)

In [ ]:
test_predictions = model.predict(X_test).flatten()
pred = pd.DataFrame({'test': y_test})
pred['predictions'] = test_predictions

fig = px.scatter(pred, 'test', 'predictions')
fig.add_trace(go.Scatter(x=[0, df1.TARGET.max()], y=[0, df1.TARGET.max()], mode='lines'))
fig.update_layout(width=1000, height=500, title='', xaxis_title='y_true', yaxis_title='y_pred')
fig.show()

In [ ]:
sorted = pred.sort_values(by='test')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(0, y_test.shape[0], 1), y = sorted['test'], mode='lines', name='y_true'))
fig.add_trace(go.Scatter(x = np.arange(0, y_test.shape[0], 1), y =sorted['predictions'], mode='lines', name='y_pred'))
fig.update_layout(width=1000, height=500, title='', xaxis_title='sample', yaxis_title='Close')
fig.show()

By Time

In [ ]:
X = df2.drop('TARGET', axis=1).copy()
y = df2['TARGET'].copy()

length = df2.shape[0]
range = int(round(0.8 * length, 0))

X_train = X[:range]
X_test = X[range:]
y_train = y[:range]
y_test = y[range:]

features = ['Day', 'Month', 'Year', 'RSI', 'MFI', 'CMO', 'UO', 'ADX']
scaler = MinMaxScaler()
scaler.fit(X_train[features])
X_train[features] = scaler.transform(X_train[features])
X_test[features] = scaler.transform(X_test[features])

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

y_train = y_train.reshape(y_train.shape[0], 1, 1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def build_model_rnn():
    lstm = Sequential()
    lstm.add(LSTM(32, input_shape=(1, X_train.shape[2]), activation='relu', return_sequences=False))
    lstm.add(Dense(1))
    lstm.compile(optimizer='adam', 
                 loss='mape',
                 metrics=['mape', 'mse'])

    return lstm

In [ ]:
model = build_model_rnn()
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_17 (LSTM)              (None, 32)                7168      
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#es = EarlyStopping(monitor='val_mape', mode='min', verbose=0, patience=2)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.25, verbose=0, batch_size=32)#, callbacks=[es])

metrics = pd.DataFrame(history.history)
metrics.iloc[-1:]

,loss,mape,mse,val_loss,val_mape,val_mse
19,0.65633,0.65633,2.969281,0.807085,0.807085,159.029037


In [ ]:
y_pred = model.predict(X_test).flatten()
y_true = np.copy(y_test)

y_pred = y_pred.reshape(y_pred.shape[0])

print('mape', mean_absolute_percentage_error(y_true, y_pred))
print('rmse', mean_squared_error(y_true, y_pred, squared=False))

mape 0.008911855458488549
rmse 27.895083184504447


In [ ]:
plot_history(history)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(0, y_test.shape[0], 1), y = y_true, mode='lines', name='y_true'))
fig.add_trace(go.Scatter(x = np.arange(0, y_test.shape[0], 1), y = y_pred, mode='lines', name='y_pred'))
fig.update_layout(width=1000, height=500, title='', xaxis_title='sample', yaxis_title='Close')
fig.show()

Time Series Split

In [ ]:
X = df2.drop('TARGET', axis=1).copy()
y = df2['TARGET'].copy()

features = ['Day', 'Month', 'Year', 'RSI', 'MFI', 'CMO', 'UO', 'ADX']
normal_inputs = X
normal_inputs[features] = (X[features] - X[features].min()) / (X[features].max() - X[features].min())
targets = y

timesplit= TimeSeriesSplit(n_splits=10)
for train_index, test_index in timesplit.split(normal_inputs):
        X_train, X_test = normal_inputs[:train_index.shape[0]], normal_inputs[train_index.shape[0]: (train_index.shape[0]+test_index.shape[0])]
        y_train, y_test = targets[:train_index.shape[0]].values.ravel(), targets[train_index.shape[0]: (train_index.shape[0]+test_index.shape[0])].values.ravel()

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

y_train = y_train.reshape(y_train.shape[0], 1, 1)

In [ ]:
def build_model_rnn():
    lstm = Sequential()
    lstm.add(LSTM(128, input_shape=(1, X_train.shape[2]), activation='relu', return_sequences=False))
    lstm.add(Dense(1))
    lstm.compile(optimizer='adam', 
                 loss='mape',
                 metrics=['mape', 'mse'])

    return lstm

In [ ]:
model = build_model_rnn()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               77824     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 77,953
Trainable params: 77,953
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=0, patience=10)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.25, verbose=0, batch_size=32)#, callbacks=[es])

metrics = pd.DataFrame(history.history)
metrics.iloc[-1:]

,loss,mape,mse,val_loss,val_mape,val_mse
19,0.662262,0.662262,12.082292,1.296076,1.296076,420.560516


In [ ]:
y_pred = model.predict(X_test).flatten()
y_true = np.copy(y_test)

y_pred = y_pred.reshape(y_pred.shape[0])

print('mape', mean_absolute_percentage_error(y_true, y_pred))
print('rmse', mean_squared_error(y_true, y_pred, squared=False))

mape 0.012599603387446262
rmse 46.85320381285393


In [ ]:
plot_history(history)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.linspace(0, 1, y_test.shape[0]), y = y_true, mode='lines', name='y_true'))
fig.add_trace(go.Scatter(x = np.linspace(0, 1, y_test.shape[0]), y = y_pred, mode='lines', name='y_pred'))
fig.update_layout(width=1000, height=500, title='', xaxis_title='Time', yaxis_title='Price')
fig.show()